Connected to Python 3.13.7

# Libraries

In [38]:
from pyomo.environ import Set, Param, Var, NonNegativeReals, ConcreteModel, SolverFactory, Objective, Constraint, minimize, Binary, ConstraintList
import pandas as pd
import numpy as np
import json
import random

# Get Data

In [39]:
distance_matrix = pd.read_csv("../processed_data/distance_matrix.csv", header=None).to_numpy()
duration_matrix = pd.read_csv("../processed_data/duration_matrix.csv", header=None).to_numpy()
some_stations = pd.read_csv("../processed_data/some_stations.csv")

In [40]:
distance_matrix

array([[ 0.    , 11.4205,  8.4061, ...,  2.2988, 19.6956, 16.4128],
       [10.2816,  0.    ,  5.8469, ...,  9.098 , 17.1364,  6.3585],
       [ 9.1415,  5.1527,  0.    , ...,  7.9579, 10.9358,  7.963 ],
       ...,
       [ 2.2852,  9.8422,  6.8125, ...,  0.    , 18.102 , 14.8192],
       [20.4347, 12.964 , 11.7789, ..., 19.2512,  0.    ,  8.8207],
       [18.8965,  5.6976, 10.0991, ..., 17.713 ,  8.9202,  0.    ]],
      shape=(100, 100))

In [41]:
duration_matrix

array([[ 0.       , 14.761666 , 10.006667 , ...,  4.778333 , 22.813334 ,
        23.6      ],
       [11.818334 ,  0.       ,  8.191667 , ..., 11.906667 , 20.998333 ,
        13.405    ],
       [12.04     , 11.136666 ,  0.       , ..., 12.128333 , 15.728333 ,
        16.738333 ],
       ...,
       [ 5.5616665, 15.275    , 10.611667 , ...,  0.       , 23.418333 ,
        24.205    ],
       [22.381666 , 20.615    , 15.57     , ..., 22.47     ,  0.       ,
        11.92     ],
       [23.861666 , 12.065    , 16.273333 , ..., 23.95     , 12.183333 ,
         0.       ]], shape=(100, 100))

In [42]:
some_stations.head()

,station_name,station_id,latitude,longitude,net_flow
0,68 St & 5 Ave,2698.07,40.634160,-74.020580,0
1,West St & Liberty St,5184.08,40.711444,-74.014847,6
2,Montague St & Hicks St,4718.11,40.695224,-73.995989,-6
3,20 Ave & Shore Blvd,7391.02,40.785994,-73.915098,0
4,Frederick Douglass Blvd & Harlem River Dr,8100.01,40.830702,-73.936371,0


In [43]:
print(sum(some_stations['net_flow']))

0


# Build Model

In [44]:
model = ConcreteModel()
solver = SolverFactory('glpk')

N_BUSES = 5
MIN_BUS_CAPACITY = 10
MAX_BUS_CAPACITY = 50

M = 100

## Build the Sets

In [45]:
model.del_component('nodes')
model.del_component('source_stations')
model.del_component('sink_stations')
model.del_component('buses')
model.del_component('visit_nodes')

model.nodes = Set(initialize=some_stations['station_name'].tolist())
model.source_stations = Set(initialize=some_stations[some_stations['net_flow'] > 0]['station_name'].tolist())
model.sink_stations = Set(initialize=some_stations[some_stations['net_flow'] < 0]['station_name'].tolist())

model.visit_nodes = model.source_stations.union(model.sink_stations)
model.all_nodes = model.visit_nodes.union(Set(initialize=['Warehouse']))
# Print whether 'Warehouse' is in visit_nodes and its index in some_stations

if 'Warehouse' in model.visit_nodes:
    warehouse_idx = some_stations[some_stations['station_name'] == 'Warehouse'].index[0]
    print(f"'Warehouse' is in visit_nodes at index {warehouse_idx} in some_stations.")
else:
    print("'Warehouse' is not in visit_nodes.")

print(f"Total nodes to visit: {len(model.visit_nodes)}")
print(f"Visit Nodes: {model.visit_nodes.data()}")

model.buses = Set(initialize=[f"bus_{i}" for i in range(N_BUSES)])

'Warehouse' is not in visit_nodes.
Total nodes to visit: 64
Visit Nodes: ('West St & Liberty St', 'E 115 St & Madison Ave', '108 St & 39 Ave', 'Boston Rd & E 178 St', 'Willis Ave & E 141 St', 'E 161 St & River Ave', 'Gramercy Park N & Gramercy Park E', 'E 33 St & 1 Ave', '75 St & Northern Blvd', 'W 4 St & 7 Ave S', '52 St & 1 Ave', 'Driggs Ave & N Henry St', 'Metropolitan Ave & Bedford Ave', 'Broadway & Morris St', 'Caton Ave & St. Pauls Pl', 'Market St & Cherry St', 'Broadway & 31 St', 'Hudson St & W 13 St', '30 Ave & 12 St', 'Broadway & W 160 St', 'E 191 St & Bathgate Ave', 'W 22 St & 10 Ave', 'E 103 St & 2 Ave', '3 Ave & E 82 St', 'Central Park W & W 68 St', 'Schermerhorn St & Court St', 'E 167 St & Franklin Ave', 'Dock St & Front St', 'E 163 St & Union Ave', 'Berkeley Pl & 7 Ave', 'Brooklyn Ave & Dean St', 'E 2 St & 2 Ave', 'Montague St & Hicks St', 'Crotona Ave & E 183 St', '36 St & 4 Ave', '37 Ave & 35 St', 'W 31 St & 7 Ave', 'Smith St & 3 St', 'Fort Hamilton Pkwy & E 5 St', 'Tin

## Bus Parameters

In [46]:
bus_param_dict = {
    f"bus_{i}": {
        random.randint(MIN_BUS_CAPACITY, MAX_BUS_CAPACITY)
    } for i in range(N_BUSES)
}

model.del_component('busCapacities')

# Flatten the bus_param_dict so that keys are bus names and values are capacities (not sets)
bus_param_flat = {k: list(v)[0] for k, v in bus_param_dict.items()}

model.busCapacities = Param(model.buses, initialize=bus_param_flat, within=NonNegativeReals)


# Node Parameters

In [47]:
# We need to build the parameter dictionaries for each station which contain their supply and demand 
""" supply_param_dict = name : [supply, demand]"""

model.del_component('supply')
model.del_component('demand')

supply_param_dict = {
    row['station_name']: [
        row['net_flow'] if row['net_flow'] > 0 else 0,
        -row['net_flow'] if row['net_flow'] < 0 else 0
    ] for _, row in some_stations.iterrows()
}

model.supply = Param(
    model.visit_nodes,
    initialize={k: v[0] for k, v in supply_param_dict.items() if k in model.visit_nodes},
    within=NonNegativeReals,
    mutable=True
)


model.demand = Param(
    model.visit_nodes,
    initialize={k: v[1] for k, v in supply_param_dict.items() if k in model.visit_nodes},
    within=NonNegativeReals,
    mutable=True
)


# Decision Variables 

In [48]:
model.del_component('x')

# We need to decide if a bus k goes from node i to node j
# x_kij
model.x = Var(
    model.buses,
    model.all_nodes,
    model.all_nodes,
    within=Binary
)

# We need to model how many bikes are on board each bus k when it arrives at node i
model.del_component('y')

model.y = Var(
    model.all_nodes,
    model.buses,
    within=NonNegativeReals,
)

# # We need to know how many bikes are picked up/dropped off at each node i by bus k
# model.z = Var(
#     model.visit_nodes,
#     model.buses,
#     within=NonNegativeReals,
# )

## Objective Function

In [49]:
def min_distance_obj(model):
    return sum(
        distance_matrix[
            some_stations[some_stations['station_name'] == i].index[0],
            some_stations[some_stations['station_name'] == j].index[0]
        ] * model.x[k, i, j]
        for k in model.buses
        for i in model.all_nodes
        for j in model.all_nodes
        if i != j
    )

def min_time_obj(model):
    return sum(
        duration_matrix[
            some_stations[some_stations['station_name'] == i].index[0],
            some_stations[some_stations['station_name'] == j].index[0]
        ] * model.x[k, i, j]
        for k in model.buses
        for i in model.all_nodes
        for j in model.all_nodes
        if i != j
    )

model.del_component('objective')

model.objective = Objective(rule=min_distance_obj, sense=minimize)



## CONSTRAINTS

In [50]:
""" FLOW CONSERVATION CONSTRAINTS """
def flow_conservation_rule(model, n, k):
    inflow = sum(
        model.x[k, i, n]
        for i in model.all_nodes
        if i != n
    )
    
    outflow = sum(
        model.x[k, n, j]
        for j in model.all_nodes
        if j != n
    )
    
    return inflow == outflow


In [51]:
""" ASSIGNMENT CONSTRAINTS """
def assignment_rule(model, n):
    if n == 'Warehouse':
        return Constraint.Skip
    
    return sum(
        model.x[k, n, j]
        for k in model.buses
        for j in model.all_nodes
        if j != n
    ) == 1  # >= Multiple buses can visit the same node

In [52]:
def get_net_flow(model, node):
    if node == 'Warehouse':
        return 0
    
    return model.supply[node] - model.demand[node]

""" VEHICLE LOADING CONSTRAINT """
# Updates load for trips going to any node EXCEPT the Warehouse
def vehicle_loading_lower(model, i, j, k):
    # If we are returning to the warehouse, SKIP this rule.
    # We don't want to force on_board['Warehouse'] to equal the return load.
    if j == 'Warehouse' or i == 'Warehouse':
        return Constraint.Skip

    flow_i = get_net_flow(model, i)
    M_val = model.busCapacities[k]

    return model.y[j, k] >= model.y[i, k] + flow_i - M_val * (1 - model.x[k, i, j])

# Updates load for trips going to any node EXCEPT the Warehouse
def vehicle_loading_upper(model, i, j, k):
    # If we are returning to the warehouse, SKIP this rule.
    # We don't want to force on_board['Warehouse'] to equal the return load.
    if j == 'Warehouse' or i == 'Warehouse':
        return Constraint.Skip

    flow_i = get_net_flow(model, i)
    M_val = model.busCapacities[k]

    return model.y[j, k] <= model.y[i, k] + flow_i + M_val * (1 - model.x[k, i, j])


In [53]:
# """NODE FULLFILMENT CONSTRAINTS"""
# def node_fullfilment_rule(model, n):
#     return sum(model.z[n, k] for k in model.buses ) == get_net_flow(model, n)
    

In [54]:
# """NODE VISIT CONSTRAINTS"""
# def node_visit_rule(model, n):
#     sum(model.x[k,n,j] for k in model.buses for j in model.all_nodes if j != n) > 

In [55]:
# """ RETURN TRIP CAPACITY CONSTRAINT """
# # Ensures that when returning to the warehouse, the bus isn't overloaded
# def return_capacity_rule(model, i, k):
#     # We only care about trips ending at the Warehouse
#     if i == 'Warehouse':
#         return Constraint.Skip
        
#     flow_i = model.supply[i] if i in model.source_stations else -model.demand[i]
    
#     # If x[k, i, 'Warehouse'] == 1, then Load_i + Flow_i <= Capacity
#     return model.y[i, k] + flow_i <= model.busCapacities[k] + M*(1-model.x[k, i, 'Warehouse'])

# """ RETURN TRIP NON-NEGATIVE CONSTRAINT """
# # Ensures that when returning to the warehouse, the bus has >= 0 bikes
# def return_non_negative_rule(model, i, k):
#     # We only care about trips ending at the Warehouse
#     if i == 'Warehouse':
#         return Constraint.Skip
        
#     flow_i = model.supply[i] if i in model.source_stations else -model.demand[i]
    
#     # If x[k, i, 'Warehouse'] == 1, then Load_i + Flow_i >= 0
#     return model.y[i, k] + flow_i >= 0 - M*(1-model.x[k, i, 'Warehouse'])

In [56]:
""" VEHICLE CAPACITY CONSTRAINT """
def vehicle_capacity_rule(model, i, k):
    if i == 'Warehouse':
        return Constraint.Skip
    flow_i = get_net_flow(model, i)

    return model.y[i,k] + flow_i <= model.busCapacities[k]


In [57]:
""" SUBTOUR ELIMINATION CONSTRAINTS """
def subtour_elimination_rule(model, setA, k):
    return sum(model.x[k,i,j] for i in setA for j in setA if i != j) <= len(setA) - 1


In [58]:
""" DEPOT CONSTRAINTS """
def leave_depot_rule(model, k):
    return sum(
        model.x[k, 'Warehouse', j]
        for j in model.visit_nodes
    ) >= 1

def return_to_depot_rule(model, k):
    return sum(
        model.x[k, i, 'Warehouse']
        for i in model.visit_nodes
    ) >= 1

# Misc Functions

In [59]:
def fetch_loops(model, df, bus:int=0):
    # View Results
    print("Objective Value: ", model.objective())

    df_bus = df[df['bus'] == f'bus_{bus}']
    df_bus = df_bus[df_bus['value'] == 1]

    # Choose a random starting point (the warehouse)
    df_bus_stations = df_bus[df_bus['value'] == 1]['from'].unique()

    routes = []
    while len(df_bus_stations) > 0:
        start = np.random.choice(df_bus_stations)
        sub_route = [start]
        df_bus_stations = df_bus_stations[df_bus_stations != start]
        
        current_node = None
        
        # SAFETY: Prevent infinite loops if the solver outputs complex paths (e.g. multiple depot visits)
        steps = 0
        max_steps = len(df_bus) + 5 

        while current_node != start:
            next_nodes = df_bus[df_bus['from'] == sub_route[-1]]['to'].values
            if len(next_nodes) == 0:
                break
            
            # If there are multiple choices (e.g. at Warehouse), try to pick one that isn't the immediate predecessor to avoid ping-pong
            # or simply pick the first one.
            current_node = next_nodes[0]
            
            sub_route.append(current_node)
            if current_node in df_bus_stations:
                df_bus_stations = df_bus_stations[df_bus_stations != current_node]
            
            steps += 1
            if steps > max_steps:
                print(f"Warning: Infinite loop detected for bus {bus}. Breaking route.")
                break
        
        routes.append(sub_route)

    # for each route in the routes, take only the unique nodes
    unique_routes = [np.unique(route).tolist() for route in routes]
    
    # SAFETY CHECK: Identify the route with the Warehouse safely
    remove_route = None

    for route in unique_routes:
        if "Warehouse" in route:
            remove_route = route
            break
            
    if remove_route:
        unique_routes.remove(remove_route)

    return unique_routes


# Building and Solving

In [60]:
def build_model(model, routes_dict={}, iter=1):
    if iter == 1:
        model.del_component('flow_conservation')
        model.del_component('assignment')
        model.del_component('loading_upper')
        model.del_component('loading_lower')
        model.del_component('return_capacity')     
        model.del_component('return_non_negative') 
        model.del_component('capacity')
        model.del_component('start_depot')
        model.del_component('end_depot')


        model.flow_conservation = Constraint(
            model.all_nodes,
            model.buses,
            rule=flow_conservation_rule
        )

        model.assignment = Constraint(
            model.visit_nodes,
            rule=assignment_rule
        )

        model.loading_upper = Constraint(
            model.all_nodes, # i
            model.all_nodes, # j
            model.buses,     # k
            rule=vehicle_loading_upper
        )

        model.loading_lower = Constraint(
            model.all_nodes, # i
            model.all_nodes, # j
            model.buses,     # k
            rule=vehicle_loading_lower
        )

        # model.return_capacity = Constraint(
        #     model.visit_nodes, # i (only need to check stations returning to depot)
        #     model.buses,       # k
        #     rule=return_capacity_rule
        # )

        # model.return_non_negative = Constraint(
        #     model.visit_nodes, # i
        #     model.buses,       # k
        #     rule=return_non_negative_rule
        # )

        model.capacity = Constraint(
            model.all_nodes,
            model.buses,
            rule=vehicle_capacity_rule
        )

        model.start_depot = Constraint(
            model.buses,
            rule=leave_depot_rule
        )

        model.end_depot = Constraint(
            model.buses,
            rule=return_to_depot_rule
        )

        # Add subtour elimination constraints after the first iteration
        model.subtour_elimination_constraints = ConstraintList()

    else:
        for bus, routes in routes_dict.items():
            for route in routes:
                if len(route) >= 2:
                    model.subtour_elimination_constraints.add(
                        subtour_elimination_rule(model, route, bus)
                    )
    return model


def solve_model(model, solver, time=100, tee=False):
    solver.options['tmlim'] = time  # Time limit in seconds (e.g., 300s = 5 mins)
    solver.options['mipgap'] = 0.05 # Stop if gap is less than 5%

    sol = solver.solve(model, tee=tee)
    return sol



# model = build_model(model)
# sol = solve_model(model, solver)



# Save Results

In [61]:
def save_results(save_path = "../results/solutionIter2.csv", model = None):
    rows = []
    for key in model.x.keys():
        bus, i, j = key
        desc = model.x[key].value
        rows.append({
            "bus": bus,
            "from": i,
            "to": j,
            "value": desc
        })

    df = pd.DataFrame(rows)
    df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

    return df

# df_results = save_results(model=model)

# df_results[df_results['value'] == 1].head(10)


In [62]:
loops_bus_0 = fetch_loops(model, df_results, bus=0)


ERROR: evaluating object as numeric value: x[bus_0,West St & Liberty St,E 115
St & Madison Ave]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized VarData object x[bus_0,West St & Liberty St,E
    115 St & Madison Ave]


ValueError: No value for uninitialized VarData object x[bus_0,West St & Liberty St,E 115 St & Madison Ave]

# Iterative Solver

In [ ]:
from pyomo.environ import ConstraintList

def iterative_solve(model, solver):
    routes_dict = {i: [[None],[None]] for i in model.buses}
    iter = 0

    while any(len(routes) > 0 for routes in routes_dict.values()):
        iter += 1
        print(f"Iteration {iter}")

        # Build and solve the model
        model = build_model(model, routes_dict, iter)
        sol = solve_model(model, solver, time=100, tee=(iter % 5 == 0))

        # Save and load results
        df_results = save_results(save_path=f"../results/iterative2/solutionIter{iter}.csv", model=model)

        # Fetch routes for each bus
        for bus in model.buses:
            unique_routes = fetch_loops(model, df_results, bus=int(bus.split('_')[1]))
            print(unique_routes)
            routes_dict[bus] = unique_routes

            print(f"Bus {bus} # routes: {len(unique_routes)}")
    
    sol = solve_model(model, solver, time=3000)
    df_results = save_results(save_path="../results/iterative2/final_solution.csv", model=model)
    
    return df_results

In [63]:
model.del_component('subtour_elimination_constraints')
model = build_model(model)
sol = solve_model(model,solver,tee=True)
df_results = save_results("../results/solution.csv", model=model)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 100 --mipgap 0.05 --write C:\Users\matte\AppData\Local\Temp\tmp6a22uk2_.glpk.raw
 --wglp C:\Users\matte\AppData\Local\Temp\tmpct0f2usn.glpk.glp --cpxlp C:\Users\matte\AppData\Local\Temp\tmpzel54kys.pyomo.lp
Reading problem data from 'C:\Users\matte\AppData\Local\Temp\tmpzel54kys.pyomo.lp'...
C:\Users\matte\AppData\Local\Temp\tmpzel54kys.pyomo.lp:351927: warning: lower bound of variable 'x21192' redefined
C:\Users\matte\AppData\Local\Temp\tmpzel54kys.pyomo.lp:351927: warning: upper bound of variable 'x21192' redefined
41679 rows, 21440 columns, 184640 non-zeros
21120 integer variables, all of which are binary
373047 lines were read
Writing problem data to 'C:\Users\matte\AppData\Local\Temp\tmpct0f2usn.glpk.glp'...
310236 lines were written
GLPK Integer Optimizer 5.0
41679 rows, 21440 columns, 184640 non-zeros
21120 integer variables, all of which are binary
Preprocessing...
PROBLEM HAS NO PRIMAL FEASIBLE

In [ ]:
model.del_component('subtour_elimination_constraints')
iterative_solve(model, solver)

Iteration 1
Results saved to ../results/iterative2/solutionIter1.csv
Objective Value:  260.85779999999994
[['62 St & Northern Blvd', '75 St & Northern Blvd', 'Amsterdam Ave & W 156 St', 'Aqueduct Ave & North St', 'Boston Rd & E 178 St', 'Broadway & W 160 St', 'Broadway & W 192 St', 'Driggs Ave & N Henry St', 'E 103 St & 2 Ave', 'E 103 St & Lexington Ave', 'E 161 St & River Ave', 'E 2 St & Ave C', 'Greenwich Ave & 8 Ave', 'Hudson St & W 13 St', 'Manhattan Ave & Devoe St', 'Metropolitan Ave & Meeker Ave', 'Tinton Ave & E 165 St', 'W 22 St & 10 Ave', 'Warehouse'], ['24 Ave & 26 St', '62 St & Northern Blvd', '75 St & Northern Blvd', 'Amsterdam Ave & W 156 St', 'Aqueduct Ave & North St', 'Boston Rd & E 178 St', 'Broadway & 31 St', 'Broadway & W 160 St', 'Broadway & W 192 St', 'Bulova Ave & Brooklyn Queens Expressway W', 'Crotona Ave & E 183 St', 'Ditmars Blvd & 76 St', 'Driggs Ave & N Henry St', 'E 103 St & 2 Ave', 'E 103 St & Lexington Ave', 'E 115 St & Madison Ave', 'E 161 St & River Ave'

# View Results

In [ ]:
def view_results(model,df):
    # View Results
    print("Objective Value: ", model.objective())
    for bus in range(N_BUSES):
        print(f"The route taken by bus_{bus}:\n")

        df_bus = df[df['bus'] == f'bus_{bus}']
        df_bus = df_bus[df_bus['value'] == 1]

        # Choose a random starting point (the warehouse)
        df_bus_stations = df_bus[df_bus['value'] == 1]['from'].unique()

        routes = []
        while len(df_bus_stations) > 0:
            start = np.random.choice(df_bus_stations)
            sub_route = [start]
            df_bus_stations = df_bus_stations[df_bus_stations != start]
            
            current_node = None
            while current_node != start:
                next_nodes = df_bus[df_bus['from'] == sub_route[-1]]['to'].values
                if len(next_nodes) == 0:
                    break
                current_node = next_nodes[0]
                sub_route.append(current_node)
                if current_node in df_bus_stations:
                    df_bus_stations = df_bus_stations[df_bus_stations != current_node]
            
            routes.append(sub_route)
        
            print(" -> ".join(sub_route), "\n")

    visited_paths = df[df['value'] == 1][['from', 'to']]
    print("The number of unique nodes visited is:", len(visited_paths['from'].unique()))
    
    # random_station = df_bus_stations.sample(1).values[0]
    # random_station.head()
    
    # route = ['Warehouse']
    # current_node = df_bus[(df_bus['from'] == 'Warehouse')]['to'].values[0]
    # route.append(current_node)
    
    # while current_node != 'Warehouse':
    #     next_node = df_bus[(df_bus['from'] == current_node)]['to'].values
    #     if len(next_node) == 0:
    #         break
    #     current_node = next_node[0]
    #     route.append(current_node)
    
    # print(route)
view_results(model, pd.read_csv("../results/iterative/solutionIter15.csv"))
print("Number of nodes to visit:", len(model.visit_nodes))


Objective Value:  157.4488
The route taken by bus_0:

Warehouse -> Central Park W & W 68 St -> W 22 St & 10 Ave -> Hudson St & W 13 St -> Greenwich Ave & 8 Ave -> Warehouse 

37 Ave & 35 St -> 24 Ave & 26 St -> 30 Ave & 12 St -> Broadway & 31 St -> 37 Ave & 35 St 

E 2 St & Ave C -> Columbia St & Rivington St -> E 2 St & Ave C 

The route taken by bus_1:

Metropolitan Ave & Bedford Ave -> Market St & Cherry St -> E 2 St & 2 Ave -> Warehouse -> W 4 St & 7 Ave S -> Bank St & Washington St -> West St & Liberty St -> Broadway & Morris St -> Smith St & 3 St -> Berkeley Pl & 7 Ave -> Prospect Pl & Vanderbilt Ave -> Brooklyn Ave & Dean St -> Monroe St & Marcus Garvey Blvd -> Manhattan Ave & Devoe St -> Metropolitan Ave & Bedford Ave 

Aqueduct Ave & North St -> E 176 St & Clinton Ave -> Boston Rd & E 178 St -> E 167 St & Franklin Ave -> Franklin Ave & E 169 St -> Tinton Ave & E 165 St -> Willis Ave & E 141 St -> E 161 St & River Ave -> E 163 St & Union Ave -> E 115 St & Madison Ave -> Broadwa